# AKI Seminar2 Demo
## Finetuning of LLaMA

by Syon Kadkade


Table of contents

> [Install packages](#install)   
> [Import libaries](#imports)   
> [Lorem Ipsum]()


In [1]:
!nvidia-smi

Fri Jan 19 09:55:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

--------------
<a id="install"></a>
### Install packages[Emoji]

**Description**:   
lorem ipsum

In [1]:
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install datasets --quiet
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00


---------------
<a id="imports"></a>
### Import libaries [Emoji]
**Description**:   
Load all necessary libaries.

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import load_dataset
from peft import LoraConfig, TaskType
from peft import get_peft_model
from accelerate import Accelerator
#from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, DataCollatorForSeq2Seq, GenerationConfig, TrainingArguments, Trainer

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

-----------------
### Load LLaMA-7B-Model[Emoji]

**Description**:  
lorem ipsum dolor sit amet

**Resources**:
- [Tutorial](#https://www.youtube.com/watch?v=t68IV5t5UOA)
- [Hugging Face: Transformer Tutorial](#https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt)
- [Hugging Face: LLaMA-7B-Model](https://huggingface.co/docs/transformers/main/model_doc/llama)
- [Hugging Face: LLaMA weights](https://huggingface.co/luodian/llama-7b-hf)
- [Hugging Face: 7B Weights](#https://huggingface.co/huggyllama/llama-7b)

**Note**: I use a model that has the weights in it and we introduce these into the actual LLaMA model. Normally you have to request the weights from Meta AI by filling out a form. I have filled it out several times but there is no response from them.

In [4]:

#MODEL_NAME = "Enoch/llama-7b-hf"
#MODEL_NAME = "baffo32/decapoda-research-llama-7B-hf"
#PEFT_MODEL_NAME = "tloen/alpaca-lora-7b"
#PEFT_MODEL_NAME = "dominguesm/alpaca-lora-ptbr-7b"
#PEFT_MODEL_NAME = "Eterna2/alpaca-lora-7b"

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [263]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, load_in_8bit=True, device_map="auto")


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [209]:
model.config.use_cache = False

In [210]:
model.config.use_cache = False
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': True,
 'load_in_4bit': False,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'fp4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float32'}

In [211]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM

)

In [212]:
lora_model = get_peft_model(model=model, peft_config=lora_config)
lora_model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


----------------
### Load Alpaca dataset
**Rescources**:
- [HuggingFace: datasets tutorial](#https://huggingface.co/docs/datasets/tutorial)
- [HuggingFace: vicgale/alpaca-gpt4 dataset](#https://huggingface.co/datasets/vicgalle/alpaca-gpt4)

In [8]:
#dataset = load_dataset("vicgalle/alpaca-gpt4")
dataset = load_dataset("imdb")

In [214]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [241]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [242]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [234]:
type(dataset)

datasets.dataset_dict.DatasetDict

-------------
### Data Preprocessing

In [235]:
def convert_score(score):
  if score == 1:
    return "Positive"

  elif score == 0:
    return "Negative"

In [243]:
def create_prompt(prompt):
  s = convert_score(prompt["label"])
  template= f""" Below are some
  ### INSTRUCTION: Detect the sentiment of the review.
  ### INPUT: [INPUT]
  ### RESPONSE: [OUTPUT]
  """

  template = template.replace("[INPUT]", prompt["text"])
  template = template.replace("[OUTPUT]", s)
  return template

In [244]:
def tokenize(prompt):
    results = tokenizer(prompt, max_length=256, truncation=True, padding=False, return_tensors=None)

    if (results["input_ids"][-1] != tokenizer.eos_token_id and len(results["input_ids"]) < 256):
        results["input_ids"].append(tokenizer.eos_token_id)
        results["attention_mask"].append(1)

    results["labels"] = [str_to_int[label] for label in prompt["labels"]]
    #results["labels"] = results["input_ids"].copy()

    return results

In [245]:
def prepare_dataset(prompt):
  prepared_prompt = create_prompt(prompt)
  tokenized_prompt = tokenize(prepared_prompt)
  return tokenized_prompt

In [247]:
train_X = dataset["train"].map(prepare_dataset)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
train_X

In [139]:
train_X = train_X.remove_columns(dataset["train"].column_names)

ValueError: Column name instruction not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']

In [ ]:
train_X["labels"]

In [120]:
test_X = dataset["test"].map(prepare_dataset)

Map:   0%|          | 0/13001 [00:00<?, ? examples/s]

In [248]:
test_X

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 13001
})

In [121]:
test_X = test_X.remove_columns(dataset["train"].column_names)

----------------
### Prepare Trainer for finetuning
Do it via Trainer API or with own training pipeline

In [4]:
MODEL_NAME = 'huggyllama/llama-7b'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = (0)
tokenizer.padding_side = "left"

In [5]:
dataset = load_dataset("imdb")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [7]:
tokenized_imdb = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, id2label=id2label, label2id=label2id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at huggyllama/llama-7b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM

)

In [12]:
lora_model = get_peft_model(model=model, peft_config=lora_config)
lora_model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,611,546,112 || trainable%: 0.06343907958816639


In [15]:
model.config.use_cache = False
#model.config.quantization_config.to_dict()

In [15]:
#gradient_accumulation_steps = Bacth_size // Micro_Batch_Size
"""
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps= (128 // 4),
    warmup_steps=100,
    max_steps=300,
    learning_rate=3e-4,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    eval_steps=50,
    save_steps=50,
    output_dir="./content/experiments",
    save_total_limit=3,
    load_best_model_at_end=True
)
"""

In [15]:
  training_args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir="./content/experiments",
    load_best_model_at_end=True
)

In [14]:
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, model=MODEL_NAME)


In [16]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

----------------
### Finetune Model

In [17]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 352.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 160.81 MiB is free. Process 20781 has 39.40 GiB memory in use. Of the allocated memory 38.04 GiB is allocated by PyTorch, and 886.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
lora_model.save_pretrained("./content/")

In [ ]:
tokenizer.save_pretrained("./content/")

----------------
### Example Prompting

In [154]:
def example_prompt(data):
  template = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: [INSTRUCTION] ### Input: [INPUT] ### Response:"""

  template = template.replace("[INSTRUCTION]", data["instruction"])
  template = template.replace("[INPUT]", data["input"])
  return template

In [152]:
first_data = dataset["train"][1]
first_data

{'instruction': 'What are the three primary colors?',
 'input': '',
 'output': 'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary colors?\n\n### Response:\nThe three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).'}

In [155]:
#text = "Write me python code"
text_prepared = example_prompt(first_data)
text_prepared

'Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What are the three primary colors? ### Input:  ### Response:'

In [156]:
inputs = tokenizer(text_prepared, return_tensors="pt") #Here we don't use padding because it is ineffecient. Better use after batching.

In [142]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    repetition_penalty=1.1
)

In [157]:
generation_output = lora_model.generate(**inputs,
    generation_config=generation_config,
    return_dict_in_generate=False,
    output_scores=False,
    max_new_tokens=256
)

In [158]:
decoded_string = tokenizer.decode(generation_output[0])
print(decoded_string)

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What are the three primary colors? ### Input:  ### Response: 1. Red, yellow and blue. 2. Red, green and blue. 3. Red, blue and yellow.
What are the three primary colors?</s>
